# Softmax 回归的从零开始实现

这一节我们来动手实现 Softmax 回归。首先导入本节实现所需的包或模块。

In [1]:
%matplotlib inline
import gluonbook as gb
from mxnet import autograd, nd

## 获取和读取数据¶

我们将使用 Fashion-MNIST 数据集，并设置批量大小为 256。

In [2]:
batch_size = 256
train_iter, test_iter = gb.load_data_fashion_mnist(batch_size)

## 初始化模型参数

跟线性回归中的例子一样，我们将使用向量表示每个样本。已知每个样本输入是高和宽均为 28 像素的图像。模型的输入向量的长度是  28×28=784 ：该向量的每个元素对应图像中每个像素。由于图像有 10 个类别，单层神经网络输出层的输出个数为 10。所以 Softmax 回归的权重和偏差参数分别为  784×10  和  1×10  的矩阵。

In [3]:
num_inputs = 784
num_outputs = 10

W = nd.random.normal(scale=0.01, shape=(num_inputs, num_outputs))
b = nd.zeros(num_outputs)

W, b

(
 [[ 0.01163079  0.00483805  0.00299563 ... -0.0235563   0.0054144
    0.02678507]
  [ 0.01254634 -0.00548774 -0.00681064 ... -0.02757963  0.0107628
   -0.00614133]
  [ 0.01830765 -0.01146806  0.00053838 ...  0.00201315  0.00350055
    0.00536052]
  ...
  [-0.00507716 -0.00258162 -0.00671208 ...  0.00318219 -0.00042557
   -0.00010889]
  [-0.00392015 -0.0097427   0.00342802 ... -0.01475318  0.01448503
    0.00457099]
  [-0.01297282 -0.01351191 -0.00353406 ...  0.01087382 -0.00354935
   -0.00361599]]
 <NDArray 784x10 @cpu(0)>, 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 <NDArray 10 @cpu(0)>)

同之前一样，我们要对模型参数附上梯度。

In [4]:
W.attach_grad()
b.attach_grad()

## 实现 Softmax 运算¶

在介绍如何定义 Softmax 回归之前，我们先描述一下对如何对多维 NDArray 按维度操作。

在下面例子中，给定一个 NDArray 矩阵X。我们可以只对其中同一列（axis=0）或同一行（axis=1）的元素求和，并在结果中保留行和列这两个维度(keepdims=True)。

In [10]:
X = nd.array([[1, 2, 3], [4, 5, 6]])
X.sum(axis=0, keepdims=True), X.sum(axis=1, keepdims=True)

(
 [[5. 7. 9.]]
 <NDArray 1x3 @cpu(0)>, 
 [[ 6.]
  [15.]]
 <NDArray 2x1 @cpu(0)>)

(2, 3)